<h1 style="color:#FF0000">Set-up</h1>


### Import libraries
- Imports libraries needed for this notebook

In [ ]:
# pip install chart_studio

In [ ]:
import pandas as pd
from datetime import datetime as dt
import urllib.request
import json
import yfinance as yf
import time
from tqdm import tqdm
from requests.exceptions import HTTPError, RequestException
import plotly.express as px

In [ ]:
# Import and setup for plotly in Colab
import chart_studio
import chart_studio.plotly as py # for plotting graphs
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

# Enable displaying pandas data frames as interactive tables by default
from google.colab import data_table # so tables don't suck
data_table.enable_dataframe_formatter()

- Pick your start and end dates, <br>
- whether you want the peer data or the company data,<br>
- and what threshold you want for percentage presence over time of each company's stock within an index <br>
- choose your file <br>
- and your index <br>
- download_lag is the pause in seconds between evaluating each ticker, i recommend 0.5 to prevent yahoo throttling your access

In [ ]:
start ='2019-12-01' #@param {type:string} ['2014-01-01','2019-12-01']
end = 'today' #@param {type:string} ['2019-11-01','today']
benchmark= True #@param {type:'boolean'}
threshold=90 #@param {type:'integer'}
chosen_ticker_file = 'S&P500tickers.csv' #@param {type:string} ['S&P500tickers.csv','FTSE100.csv']
fin_index= 'SP500' #@param {type:string} ['SP500','FTSE100']
download_lag= 0.25 #@param {type: "slider", min: 0, max: 2, step: 0.01}


if end == 'today':
  end = None
# when slicing with None it goes right up to the end of the dataframe

#create time variable to name dataset
if start == '2014-01-01':
  if end==None:
    time_range='all'
  elif end=='2019-11-01':
    time_range='before'
else:
    time_range='after'

#create data variable to name dataset
if benchmark==True:
    datatype='peerGroup'
else:
    datatype='ticker'

<h1 style="color:#FF0000">Tickers list</h1>

### Create tickers list function
- Takes a CSV for an index fund with a list of dates, each with a corresponding list of tickers that were in an index on that date
- Also takes a threshold percentage time (out of 100) that a stock has to have been on the index (default 90)
- Returns a list of tickers which have been on that index a specified percentage of the time between 2014 and 2025

In [ ]:
 # code I borrowed from: https://medium.com/@rganesh0203/different-ways-to-count-duplicates-in-list-python-in-worst-to-best-complexities-8a23c3ebd9c4
# this code just counts how many times an element occurs in a list and puts that in a dictionary with key as element and value as number of occurences
def count_duplicates_dict(lst):
        duplicates = {}
        for item in lst:
            if item in duplicates:
                duplicates[item] += 1
            else:
                duplicates[item] = 1
        return {key: value for key, value in duplicates.items() if value > 1}

In [ ]:
def tickers_func(file, threshold=90):
    strongest=[]      #creates an empty list
    df=pd.read_csv(file)      #read csv file

    df['date']=pd.to_datetime(df['date'])      #set date column as datetime object
    df=df[df['date']>'2014-01-01']      #we only want data pre 2014
    df_list=list(df['tickers'])      #create a list, each element is a string that contains all the stock tickers for that date
    total=len(df_list)      #gets total number of timestamp rows

    for index,value in enumerate(df_list):      #iterates through df_list
        strongest.extend(value.split(','))      #splits the strings of tickers into lists of strings of single tickers and adds all those lists into one big list
    dict_best_tickers=count_duplicates_dict(strongest)      #uses the count duplicates function to count how many times a ticker shows up in the list

    for i in dict_best_tickers:      #iterates through dictionary and replaces count of occurences with percentage of occurences
        dict_best_tickers[i]=round(100*dict_best_tickers[i]/total,2)
    sorted_dict = dict(sorted(dict_best_tickers.items(), key=lambda item: item[1],reverse=True))      #sorts by percentage of occurences

    final_dict=dict()      #create empty dictionary

    for i in sorted_dict:      #iterate through dictionary, adding values which exceed the occurence threshold to the empty dictionary
        if sorted_dict[i]>threshold:
            final_dict[i]=sorted_dict[i]

    tickers=list(final_dict.keys())      #gets the keys i.e. tickers of the dictionary and puts them in a list

    return tickers      #exports the list



```
# This is formatted as code
```

### S&P 500 tickers list

### These will need to be changed if the index is changed (has been done manually)

In [ ]:
tickers=tickers_func(chosen_ticker_file, threshold=threshold)      #calls the function above

In [ ]:
# used claude anthropic to sort failing tickers into tickers to change and tickers to remove
change_dict= {
        'FB': 'META',
        'ANTM': 'ELV',   #updated tickers
        'BLL': 'BALL',
        'DISCA': 'WBD',
        'CTL': 'LUMN',
        'MYL': 'VTRS',
        'FISV': 'FI',
        'COG': 'CTRA',
        'BF.B':'BF-B',
        'BRK.B':'BRK-B',



        }
remove_list =[
        'A',
        'NLSN',
        'CERN',
        'XLNX',
        'KSU',
        'ALXN',  #comapanies that have gone private
        'VAR',
        'TIF',
        'NBL',
        'ETFC',
        'ADS',
        'AGN',
        'FLIR',
        'GPS',
        'MRO',
        'PXD',
        'PKI',
        'CTXS',
        'PBCT',
        'ABC',
        'GOOG'
        ]

In [ ]:
for _index, _value in change_dict.items():
  tickers[tickers.index(_index)]=_value
for _value in remove_list:
  tickers.remove(_value)

### FTSE 100 stock data

<h1 style="color:#FF0000">Getting tables of historical ESG data using tickers list</h1>

### Define esg score function
- This function takes 1 ticker e.g. 'AAPL', and start/end dates in a yyyy-mm-dd format
- Then returns 2 things:
- Firstly a dataframe with ESG, E, S and G scores over time
- Secondly a dictionary with tickers as keys and percentage of values ffilled as values
- When benchmark = True this function is exactly the same except it gives ESG/E/S/G data for the peergroup of tickers, acting as a benchmark score for that ticker, the benchmark group is found in yf.Ticker('E.g. AAPL').sustainability['peerGroup'] or by scraping the same url as below

-  adapted from code by jrpars1 on: https://stackoverflow.com/questions/75957370/invalid-synthax-in-df-while-fetching-esg-data-from-yahoo-finance

In [ ]:
def esg_score_func(ticker,start=None,end=None,benchmark=False):      #ticker string, start date, end date (both as yyyy-mm-dd)

    url = f"https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={ticker}"      #url for get request
    connection = urllib.request.urlopen(url)      #perform get request

    data = connection.read()      #read request
    data_source = json.loads(data)      #loads as a json

    if benchmark==False:
        data = data_source["esgChart"]["result"][0]["symbolSeries"]                      #unpacks json
        m=ticker

    else:
        data = data_source["esgChart"]["result"][0]['peerSeries']                   #if benchmark = True returns sector scores for peerGroup to use for benchmarking
        m = data_source['esgChart']['result'][0]['peerGroup']


    data = pd.DataFrame(data)      #turns it into dataframe              #if benchmark = False returns ticker scores
    data.rename(columns={'esgScore':'esgScore'+ m,'governanceScore':'governanceScore' + m, 'environmentScore':'environmentScore'+ m,'socialScore':'socialScore'+ m}, inplace=True)
    data["timestamp"] = pd.to_datetime(data["timestamp"], unit="s")      #change dtype of timestamp to datetime object
    data.set_index('timestamp',drop=True,inplace=True)      #turns timestamp column to index

    data=data.loc[start:end]      #cuts down dataframe to data between the start and end dates
    data.ffill(inplace=True)      #forwardfills missing data


    return(data)

### EITHER: Create historical ESG table for list of companies (benchmark=False)

- merges all the ESG and E/S/G values for each ticker to a master table
- master table has time index created in previous cell i.e. the rows represent time
- each merge on master table adds four columns each with the four scores of a specific ticker e.g. esg_aapl,e_aapl,s_aapl,g_aapl
- Creates ESG_table and na_dict

### OR: Create historical ESG data for peer_group of companies (benchmark =True)

- Create similar ESG|E/S/G tables but the scores are for the peerGroup of the ticker not the ticker itself
- We can use these to benchmark tickers compared to their own sector and measure peerGroup performance
- Code is exactly the same but we use esg_score_func(benchmark=True) instead
- Creates ESG_peer_table (the variables won't mention peer but the dataset would)

In [ ]:
time_range

'after'

In [ ]:
# iterates through all tickers in ticker list
# merges all the ESG and E/S/G values for each ticker to a master table
# master table has time index created in previous cell i.e. the rows represent time
# each merge on master table adds four columns each with the four scores of a specific ticker


esg_=esg_score_func(tickers[0],start=start,end=end,benchmark=benchmark) #sets up initial dataframe
ESG_table=esg_


for ticker in tqdm(tickers[1:]):      #tqdm gives a progress bar for iterations, we're iterating through tickers list

    time.sleep(download_lag)      #this stops me from getting banned from using yahoo finance by limiting request rate with a pause


    esg_=esg_score_func(ticker,start=start,end=end,benchmark=benchmark)      #get esg data for each ticker for pre-2019
    ESG_table=ESG_table.merge(esg_,how='outer',suffixes=('','_extra'),left_index=True,right_index=True)      #merge ticker esg data onto master table

  2%|▏         | 8/348 [00:02<01:47,  3.17it/s]<ipython-input-317-1846a721021c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ffill(inplace=True)      #forwardfills missing data
100%|██████████| 348/348 [01:52<00:00,  3.10it/s]


In [ ]:
print(time_range)

after


In [ ]:
ESG_table.tail(15)

,esgScoreTechnology Hardware,governanceScoreTechnology Hardware,environmentScoreTechnology Hardware,socialScoreTechnology Hardware,esgScorePharmaceuticals,governanceScorePharmaceuticals,environmentScorePharmaceuticals,socialScorePharmaceuticals,esgScoreHealthcare,governanceScoreHealthcare,...,environmentScoreAutomobiles_extra,socialScoreAutomobiles_extra,esgScoreRetailing_extra,governanceScoreRetailing_extra,environmentScoreRetailing_extra,socialScoreRetailing_extra,esgScoreEnergy Services_extra,governanceScoreEnergy Services_extra,environmentScoreEnergy Services_extra,socialScoreEnergy Services_extra
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-01,12.530000,5.330000,0.860000,6.340000,26.205000,8.768000,2.304000,15.458000,20.265000,6.612500,...,6.886216,10.213784,25.256667,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000
2023-12-01,12.530000,5.330000,0.860000,6.340000,26.205000,8.768000,2.304000,15.458000,20.265000,6.612500,...,6.886216,10.213784,25.256667,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000
2024-01-01,21.873333,5.330000,0.860000,6.340000,29.335000,8.768000,2.304000,15.458000,26.996000,6.612500,...,6.886216,10.213784,21.988000,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000
2024-02-01,21.873333,5.330000,0.860000,6.340000,29.335000,8.768000,2.304000,15.458000,26.996000,6.612500,...,6.886216,10.213784,21.988000,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000
2024-03-01,21.873333,5.330000,0.860000,6.340000,29.335000,8.768000,2.304000,15.458000,26.996000,6.612500,...,6.886216,10.213784,21.988000,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000
2024-04-01,21.873333,5.330000,0.860000,6.340000,29.335000,8.768000,2.304000,15.458000,26.996000,6.612500,...,6.886216,10.213784,21.988000,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000
2024-05-01,21.873333,5.330000,0.860000,6.340000,29.335000,8.768000,2.304000,15.458000,26.996000,6.612500,...,1.400000,8.980000,21.988000,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000
2024-06-01,21.873333,5.330000,0.860000,6.340000,29.335000,8.768000,2.304000,15.458000,26.996000,6.612500,...,1.400000,8.980000,21.988000,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000
2024-07-01,12.920000,5.330000,0.860000,6.340000,29.335000,8.768000,2.304000,15.458000,26.020000,6.612500,...,1.400000,8.980000,18.866667,4.773600,2.730800,7.848800,23.740000,7.310000,8.620000,7.810000


In [ ]:
c=0
for i in ESG_table.columns: #drops duplicate columns
    if '_extra' in i:
        try:
            ESG_table=ESG_table.drop(i,axis=1)
        except:
            c+=1
print(c)

if time_range=='after':
    ESG_table.loc['2019-12-01']=ESG_table.loc['2020-01-01']
else:
    pass

1080


<h1 style="color:#FF0000">Create metadata tables</h1>

In [ ]:
list_metrics=list(yf.Ticker('AAPL').sustainability.index) #creates list of possible information we can get from sustainability attribute
columns=['Symbol','Name_short','Name_long','Zip_code','City','State','Country','Website','Industry','Sector','Full-time_Employees']
#metrics we've chosen to take from yf.Ticker(ticker string i.e. "AAPL").info
columns.extend(list_metrics) #add on sustainability metrics
metadata=pd.DataFrame(columns=columns)

In [ ]:
# creates a dataframe called metadata with each row as a ticker and its associated metadata
# using lots of try/except pairs to handle non-US companies not having state or zip code
for ticker in tqdm(tickers): #tqdm makes a progress bar, iterate over tickers list, choose tickers to choose index

    time.sleep(download_lag) #prevents request speed throttling for having too fast of a rate

    try:


        tick=yf.Ticker(ticker) #initialises Ticker class
        stock=tick.info #gets info attr
        esg = tick.sustainability #gets sustainability attr


        try:
            has_zip=stock['zip'] #zip and state handling
            has_state=stock['state']
        except:
            has_zip=None
            has_state=None


        try:
            has_employees=stock['fullTimeEmployees'] #missing employee data handling
        except:
            has_employees=None


            stock_row=[stock['symbol'],stock['shortName'],stock['longName'],has_zip,stock['city'],has_state,stock['country'],stock['website'],stock['industry'],stock['sector'],has_employees]
            stock_row.extend([esg.loc[metric,'esgScores'] for metric in list_metrics]) # creates stock_row which contains all the metadata for that ticker
            stock_row=pd.DataFrame([stock_row],columns=columns) #turns stock_row into dataframe
            metadata=pd.concat([metadata, stock_row], ignore_index=True) #adds stock_row onto metadata master dataframe


    except Exception as error:
        print('^',ticker,'^')

<ipython-input-143-b587f773e6ef>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metadata=pd.concat([metadata, stock_row], ignore_index=True) #adds stock_row onto metadata master dataframe
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LB?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=LB&crumb=Sy7X4fhzMkD
<ipython-input-143-b587f773e6ef>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metadata=pd.concat([me

<h1 style="color:#FF0000">Create financial history tables</h1>

### historical financial data

In [ ]:
index_set=yf.Ticker('AAPL').history(period='max').loc[start:end].index  #creates index_set which is an index of date values

stock_value=pd.DataFrame()      #empty dataframe
dividends=pd.DataFrame()
splits=pd.DataFrame()

stock_value.index=index_set      #set index of dataframe as datetime index
dividends.index=index_set
splits.index=index_set

In [ ]:
yf.Ticker('AAPL').financials

,2024-09-30,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0,NaN
Tax Rate For Calcs,0.240912,0.147192,0.162,0.133,NaN
Normalized EBITDA,134661000000.0,125820000000.0,130541000000.0,123136000000.0,NaN
Net Income From Continuing Operation Net Minority Interest,93736000000.0,96995000000.0,99803000000.0,94680000000.0,NaN
Reconciled Depreciation,11445000000.0,11519000000.0,11104000000.0,11284000000.0,NaN
Reconciled Cost Of Revenue,210352000000.0,214137000000.0,223546000000.0,212981000000.0,NaN
EBITDA,134661000000.0,125820000000.0,130541000000.0,123136000000.0,NaN
EBIT,123216000000.0,114301000000.0,119437000000.0,111852000000.0,NaN
Net Interest Income,NaN,-183000000.0,-106000000.0,198000000.0,890000000.0
Interest Expense,NaN,3933000000.0,2931000000.0,2645000000.0,2873000000.0


In [ ]:
#iterates over tickers, gets finance data for each ticker, appends that data to its respective master dataframe of either dividends, splits or stock_value
for ticker in tqdm(tickers):

  time.sleep(download_lag) #stops it from running too fast

  ticker_=yf.Ticker(ticker) #yfinance calls ticker
  ticker_h=ticker_.history(period='max') #history attribute of ticker


  ticker_stock=ticker_h.loc[start:end,['Open','Close','Volume']] #select stock value and volume data from 2014 onwards
  ticker_stock['Mid']=(ticker_stock['Open']+ticker_stock['Close'])/2 #interpolate between open and close to produce mid value
  ticker_stock['Total_value']=ticker_stock['Volume']*ticker_stock['Mid'] #multiply volume by mid to get a representation of total value
  ticker_stock.drop(['Open','Close'],axis=1,inplace=True) #drop open and close columns
  stock_value=stock_value.merge(ticker_stock,how='outer',suffixes=('',ticker),left_index=True,right_index=True)


  ticker_div=ticker_h.loc[start:end,['Dividends']] #create df of date and dividends from 2014 onwards
  dividends=dividends.merge(ticker_div,how='outer',suffixes=('',ticker),left_index=True,right_index=True)

  ticker_splits=ticker_h.loc[start:end,['Stock Splits']] #create stock splits df from 2014 onwards
  splits=splits.merge(ticker_splits,how='outer',suffixes=('',ticker),left_index=True,right_index=True)

 89%|████████▉ | 324/365 [05:26<00:41,  1.01s/it]


KeyError: "None of [Index(['Dividends'], dtype='object')] are in the [columns]"

<h1 style="color:#FF0000">Create financial metadata tables</h1>

In [ ]:
list_metrics=list(yf.Ticker('AAPL').sustainability.index) #creates list of possible information we can get from sustainability attribute
columns=['Symbol','Name_short','Name_long','Zip_code','City','State','Country','Website','Industry','Sector','Full-time_Employees']
#metrics we've chosen to take from yf.Ticker(ticker string i.e. "AAPL").info
columns.extend(list_metrics) #add on sustainability metrics
metadata=pd.DataFrame(columns=columns)

In [ ]:
# creates a dataframe called metadata with each row as a ticker and its associated metadata
# using lots of try/except pairs to handle non-US companies not having state or zip code
for ticker in tickers: #tqdm makes a progress bar, iterate over tickers list, choose tickers to choose index

    time.sleep(download_lag) #prevents request speed throttling for having too fast of a rate

    try:


        tick=yf.Ticker(ticker) #initialises Ticker class
        stock=tick.info #gets info attr
        esg = tick.sustainability #gets sustainability attr


        try:
            has_zip=stock['zip'] #zip and state handling
            has_state=stock['state']
        except:
            has_zip=None
            has_state=None


        try:
            has_employees=stock['fullTimeEmployees'] #missing employee data handling
        except:
            has_employees=None


            stock_row=[stock['symbol'],stock['shortName'],stock['longName'],has_zip,stock['city'],has_state,stock['country'],stock['website'],stock['industry'],stock['sector'],has_employees]
            stock_row.extend([esg.loc[metric,'esgScores'] for metric in list_metrics]) # creates stock_row which contains all the metadata for that ticker
            stock_row=pd.DataFrame([stock_row],columns=columns) #turns stock_row into dataframe
            metadata=pd.concat([metadata, stock_row], ignore_index=True) #adds stock_row onto metadata master dataframe


    except Exception as error:
        print('^',ticker,'^')

<h1 style="color:#FF0000">Upload to GBQ</h1>

In [ ]:
from google.colab import auth #authenticates google colab
auth.authenticate_user()

In [ ]:
#@title access BigQuery project
project_id = 'lw-esg-project'

# Package used for interfacing w/ BigQuery from Python
from google.cloud import bigquery

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

I viciously raided this google colab notebook for code: https://colab.research.google.com/drive/1hSI1BXyCyj7viRpp1GFZqkU1qtBUd0g1?authuser=0 (produced for google by Alok Pattani I believe...)

Select the dataset, only one atm
Select output_table_id (chooses which df to upload to a table)
Select replace or append
The way I've written this that should be all you need to do

# DO NOT - I REPEAT - DO NOT: USE THIS WITHOUT CHECKING WITH AT LEAST ONE OTHER PERSON, this code can and will rewrite any work we do on bigquery (which is why it's commented out)

alternative pd: pandas_gbq.to_gbq()

In [ ]:
# Query BigQuery:
query = "SELECT * FROM `lw-esg-project.yfinance.ESG_table_before_ticker_SP500_thresh90`"

df = pd.read_gbq(query, project_id = 'lw-esg-project')

df.head()

In [ ]:
#@title upload dataframe to BigQuery Table
output_dataset_id = 'yfinance'

output_table_id = 'ESG_table' #@param{type:'string'} ['ESG_table', 'metadata', 'stock_value', 'dividends', 'splits']

replace_or_append_output = 'replace' #@param{type:'string'} ['replace', 'append']


#don't forget to change this!!!!!!!!!!!!!!!!!!!!!!!!!!!!
output_table=ESG_table #choose dataframe from dictionary created previously using selected string output_dataset_id



if output_table_id=='metadata':
    output_table_id=output_table_id
else:
    output_table_id = f"{output_table_id}_{time_range}_{datatype}"
output_table_id = f"{output_table_id}_{fin_index}_thresh{threshold}"




# Combine project and dataset
project_dataset = f"{bq_client.project}.{output_dataset_id}"

# Combine project, dataset, and table
project_dataset_table = f"{project_dataset}.{output_table_id}"

# Check to make sure output dataset exists, create it if not
try:
  bq_client.get_dataset(output_dataset_id)
  print(f"BigQuery dataset {project_dataset} exists\n")

except:
  print(f"BigQuery dataset {project_dataset} doesn't exist, so creating it\n")
  dataset = bq_client.create_dataset(bigquery.Dataset(project_dataset))

job_config = bigquery.LoadJobConfig()

# Modify job config depending on if we want to replace or append to table
if(replace_or_append_output == 'replace'):
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
else:
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

dataset_ref = bq_client.dataset(output_dataset_id)
table_ref = dataset_ref.table(output_table_id)


# Use client functionality to load BigQuery table from Pandas data frame
bq_client.load_table_from_dataframe(
  dataframe = [output_table,
  destination = table_ref,
  job_config = job_config
  ).result()

print((f"{output_table_id} ({replace_or_append_output}) to "
  f"BigQuery table {project_dataset_table}\n"))


BigQuery dataset lw-esg-project.yfinance exists

ESG_table_before_peerGroup_SP500_thresh90 (replace) to BigQuery table lw-esg-project.yfinance.ESG_table_before_peerGroup_SP500_thresh90



# Retrieve tables from bigquery

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
#@title access BigQuery project
project_id = 'lw-esg-project'

# Package used for interfacing w/ BigQuery from Python
from google.cloud import bigquery

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

In [ ]:
#@title create dataframe from BigQuery Table
%%bigquery example_df --project {project_id}
# will create pandas dataframe called example df through unexplained bigquery witchcraft
SELECT
  x
FROM
  y
ORDER BY
  z

alternatively: use pandas

In [ ]:
# Query BigQuery:
query = "SELECT * FROM `lw-esg-project.yfinance.ESG_table_before_ticker_SP500_thresh90`"

df = pd.read_gbq(query, project_id = 'lw-esg-project')

df.head()